In [ ]:
import time
import requests
import pandas as pd
from typing import List, Dict, Optional

#Function to slow down requests to avoid rate limit on the API
def fetch_with_retry(url, params, max_retries=3):
    for _ in range(max_retries):
        response = requests.get(url, params=params)
        if response.status_code == 200:
            return response.json()
        elif response.status_code == 429:
            wait_time = int(response.headers.get("Retry-After", 10))
            print(f"Rate limited. Retrying in {wait_time} seconds...")
            time.sleep(wait_time)
        else:
            print(f"Error {response.status_code}: {response.text}")
            break
    return None


def fetch_historical_weather_multiple(
    latitudes: List[float],
    longitudes: List[float],
    start_date: str,
    end_date: str,
    location_names: Optional[List[str]] = None,
) -> Dict[str, pd.DataFrame]:
    """
    Fetch historical weather data for multiple lat/lon pairs, including wind direction.

    Args:
        latitudes: List of latitudes.
        longitudes: List of longitudes.
        start_date: Start date in "YYYY-MM-DD" format.
        end_date: End date in "YYYY-MM-DD" format.
        location_names: Optional names for each location (default: "loc_0", "loc_1", ...).

    Returns:
        Dictionary of DataFrames (key: location name, value: weather data).
    """
    if len(latitudes) != len(longitudes):
        raise ValueError("Latitudes and longitudes must have the same length.")
    
    if location_names is None:
        location_names = [f"loc_{i}" for i in range(len(latitudes))]
    elif len(location_names) != len(latitudes):
        raise ValueError("Location names must match latitudes/longitudes length.")

    weather_data = {}
    base_url = "https://archive-api.open-meteo.com/v1/archive"

    for lat, lon, name in zip(latitudes, longitudes, location_names):
        params = {
            "latitude": lat,
            "longitude": lon,
            "start_date": start_date,
            "end_date": end_date,
            "hourly": [
                "temperature_2m",
                "relative_humidity_2m",
                "wind_speed_10m",
                "wind_direction_10m"
            ],
            "timezone": "auto"
        }
        
        data = fetch_with_retry(base_url, params=params,max_retries=5)

        df = pd.DataFrame(data["hourly"])
        df["location"] = name 
        weather_data[name] = df

    return weather_data

#Adjust for location of wind farm CSV
poi_df = pd.read_csv(r'C:\Users\Kavinda Nissanka\Documents\Erdos Institute\Wind Power Prediction\hydroquebec_wind_farms.csv')

# Example: Montreal (45.5017° N, 73.5673° W)
latitude = poi_df['latitude']
longitude = poi_df['longitude']
start_date = "2019-01-01"
end_date = "2024-12-31"

weather_data = fetch_historical_weather_multiple(latitude, longitude, start_date, end_date,location_names=poi_df['name'])
print(weather_data)

Rate limited. Retrying in 10 seconds...
Rate limited. Retrying in 10 seconds...
Rate limited. Retrying in 10 seconds...
Rate limited. Retrying in 10 seconds...
Rate limited. Retrying in 10 seconds...
Rate limited. Retrying in 10 seconds...
Rate limited. Retrying in 10 seconds...
Rate limited. Retrying in 10 seconds...
Rate limited. Retrying in 10 seconds...
Rate limited. Retrying in 10 seconds...
Rate limited. Retrying in 10 seconds...
Rate limited. Retrying in 10 seconds...
Rate limited. Retrying in 10 seconds...
Rate limited. Retrying in 10 seconds...
Rate limited. Retrying in 10 seconds...
Rate limited. Retrying in 10 seconds...
{'Baie-des-Sables wind farm':                    time  temperature_2m  relative_humidity_2m  wind_speed_10m  \
0      2019-01-01T00:00            -7.4                    81            17.9   
1      2019-01-01T01:00            -7.0                    80            18.9   
2      2019-01-01T02:00            -7.1                    80            21.3   
3     

In [ ]:
keys = weather_data.keys()
#Adjust path to wherever you want to save it
for key in keys:
    weather_at_station = pd.DataFrame(weather_data[key])
    weather_at_station.to_csv(r'V:\Erdos Institute Data Downloads\station_weather\{}_hourly_weather_2019-2024.csv'.format(key))